In [1]:
import urllib.request
import csv
import glob
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import StringIO
import numpy as np
import os

# Scrape Fanfooty data
This notebook is used to scrape the following data from fanfooty:

1. Current player list
2. Match stats for each player
3. Match results/fixture

## REQUIRED - Specify match IDs to scrape:

In [2]:
start_match = 8045
end_match = 8053

# start_match = 7901
# end_match = 8044

### Create folder to save all outputs

In [3]:
timestr = time.strftime("%Y%m%d-%H%M%S")
destination = "exports/scrape_{}".format(timestr)
os.mkdir(destination)

## 1. Current player list

### Save historical fanfooty player IDs csv file

In [4]:
url = "https://www.fanfooty.com.au/resource/player.php?type=all"
url_headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.80 Safari/537.36"
}
headers = "fanfooty_player_id,fanfooty_afl_player_id,fanfooty_first_name,fanfooty_surname,fanfooty_team,fanfooty_status,fanfooty_number,fanfooty_birth_date,fanfooty_height,fanfooty_weight,fanfooty_state_of_origin,fanfooty_recruited_from,fanfooty_games,fanfooty_goals"

res = requests.get(url, headers=url_headers,verify=False)
split_rows_list = res.text.split('"\r\n')
f = open('{}/fanfooty_player_ids_historical.csv'.format(destination),'w')
f.write('{}\n'.format(headers)) 
for row in split_rows_list:
    f.write('{}\n'.format(row)) 
f.close()

C:\Users\richardm\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.fanfooty.com.au'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


### Clean data:
* Remove duplicate players
* Standard team name

In [5]:
df_player_list = pd.read_csv('{}/fanfooty_player_ids_historical.csv'.format(destination), encoding = "ISO-8859-1")
df_team_names = pd.read_csv('inputs/all_sources_team_names.csv')
df_player_list['short_team'] = df_player_list['fanfooty_team'].map(dict(df_team_names[['fanfooty_team_name_mid', 'sc_team_name']].values))
df_player_list = df_player_list.dropna(subset=['fanfooty_afl_player_id'])
df_player_list = df_player_list[df_player_list['fanfooty_afl_player_id'] != 0]

remove_duplicate_players = [
    {'player_id': 271128, 'first_name': 'Danny', 'surname': 'Butcher', 'team': 'Port Adelaide'},
    {'player_id': 296214, 'first_name': 'Jake', 'surname': 'Barratt', 'team': 'GWS'},
    {'player_id': 991933, 'first_name': 'Jason', 'surname': 'Cantstandya', 'team': 'Richmond'},
    {'player_id': 992351, 'first_name': 'Bailey', 'surname': 'Williams', 'team': 'None'}
]

for player in remove_duplicate_players:
    indexNames = df_player_list[
        (df_player_list['fanfooty_afl_player_id'] == player['player_id']) & 
        (df_player_list['fanfooty_first_name'] == player['first_name']) &
        (df_player_list['fanfooty_surname'] == player['surname']) &
        (df_player_list['fanfooty_team'] == player['team'])
    ].index
    df_player_list.drop(indexNames , inplace=True)

df_player_list.to_csv('{}/fanfooty_player_ids_historical.csv'.format(destination))
df_player_list

,fanfooty_player_id,fanfooty_afl_player_id,fanfooty_first_name,fanfooty_surname,fanfooty_team,fanfooty_status,fanfooty_number,fanfooty_birth_date,fanfooty_height,fanfooty_weight,fanfooty_state_of_origin,fanfooty_recruited_from,fanfooty_games,fanfooty_goals,short_team
5,1011,980001.0,Nathan,Bassett,Adelaide,NaN,8.0,1976-12-07,190.0,88.0,SA,Norwood,210.0,25.0,ADE
6,44021,295026.0,James,Battersby,Adelaide,NaN,43.0,1995-11-07,177.0,78.0,SA,Sturt,NaN,NaN,ADE
7,46114,280974.0,Jonathon,Beech,Adelaide,NaN,1.0,1990-11-09,187.0,85.0,SA,West Adelaide,3.0,1.0,ADE
8,623,990006.0,James,Begley,Adelaide,NaN,28.0,1980-07-22,0.0,0.0,NaN,NaN,61.0,10.0,ADE
10,50128,1012807.0,Sam,Berry,Adelaide,senior,21.0,2002-02-12,181.0,81.0,VIC,Gippsland Power,30.0,5.0,ADE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5252,47084,993894.0,Lachlan,Tiziani,GWS,NaN,33.0,1997-03-13,189.0,79.0,NSW,Broken Hill,0.0,0.0,GWS
5253,42215,291928.0,Gerald,Ugle,GWS,NaN,32.0,1993-01-31,180.0,80.0,WA,Beverley,3.0,1.0,GWS
5254,39670,280109.0,Callan,Ward,GWS,senior,8.0,1990-04-10,187.0,84.0,VIC,Western Jets,264.0,130.0,GWS
5255,50159,1004530.0,Jacob,Wehr,GWS,rookie,10.0,1998-07-05,184.0,71.0,SA,Woodville-West Torrens,8.0,3.0,GWS


## 2. Match stats for each player

### Scrape match files from Fanfooty website

In [6]:
def return_list_of_urls(match_id):
    full_url_list = []
    for match in match_id:
        url = "https://www.fanfooty.com.au/live/"
        extension = ".txt"
        full_url = "{}{}{}".format(url, match, extension)
        full_url_list.append(full_url)
    return full_url_list

matches = list(range(start_match, end_match + 1))

list_of_urls = return_list_of_urls(matches)

for url in list_of_urls:
    print(url)

for url in list_of_urls:
    response = requests.get(url, headers=url_headers, verify=False)
    webContent = response.text
    filename = url[-8:]
    f = open("inputs/All Match Data/{}".format(filename), 'w', encoding="utf-8")
    f.write(webContent)
    print(filename)

https://www.fanfooty.com.au/live/8045.txt
https://www.fanfooty.com.au/live/8046.txt
https://www.fanfooty.com.au/live/8047.txt
https://www.fanfooty.com.au/live/8048.txt
https://www.fanfooty.com.au/live/8049.txt
https://www.fanfooty.com.au/live/8050.txt
https://www.fanfooty.com.au/live/8051.txt
https://www.fanfooty.com.au/live/8052.txt
https://www.fanfooty.com.au/live/8053.txt
8045.txt
8046.txt


C:\Users\richardm\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.fanfooty.com.au'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\richardm\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.fanfooty.com.au'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\richardm\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.fanfooty.com.au'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


8047.txt
8048.txt
8049.txt


C:\Users\richardm\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.fanfooty.com.au'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\richardm\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.fanfooty.com.au'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\richardm\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.fanfooty.com.au'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


8050.txt
8051.txt
8052.txt


C:\Users\richardm\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.fanfooty.com.au'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\richardm\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.fanfooty.com.au'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\richardm\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.fanfooty.com.au'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


8053.txt


### Headers of each field in match file

In [7]:
column_header_names = [
    'Fanfooty Match ID',
    'Fanfooty Match URL',
    'Round',
    'Year',
    'Player ID',
    'First Name',
    'Surname',
    'Team',
    'null',
    'DT',
    'SC',
    'null2',
    'null3',
    'null4',
    'Kicks',
    'Handballs',
    'Marks',
    'Tackles',
    'Hitouts',
    'Frees for',
    'Frees against',
    'Goals',
    'Behinds',
    'Not sure',
    'Tag',
    'Tag Notes',
    'Tag 2',
    'Tag 2 Notes',
    'null5',
    'null6',
    'null7',
    'null8',
    'Position',
    'Jumper Number',
    'null9',
    'null10',
    'null11',
    'DT own %',
    'SC own %',
    'AF own %',
    'null12',
    'AF Breakeven',
    'null13',
    'Contested Possessions',
    'Clearances',
    'Clangers',
    'Disposal efficiency',
    'Time on ground',
    'Metres gained'
    ,'Bench staus'
]

### Read match files and write to csv

In [8]:
df_fanfooty_player_raw = pd.DataFrame()
def get_number_of_lines_in_file(data):
    return len(data.split('\n'))


def get_match_id(data):
    name = data.split('\n', 1)[0]
    return name[-8:-4]


def get_url_of_match(data):
    name = data.split('\n', 1)[0]
    url = "http://live.fanfooty.com.au/game/matchcentre.html?id=" + name[-8:-4]
    return url


def get_round(data):
    line = data.split('\n', 1)[1]
    stripped_line = [x.strip() for x in line.split(',')]
    afl_round = stripped_line[4]
    return afl_round


def get_year(data):
    second_line = data.splitlines()[2]
    stripped_second_line = [x.strip() for x in second_line.split(',')]
    afl_year = stripped_second_line[1]
    return afl_year


def get_match_data_list():
    data_list = []
    path = "inputs/All Match Data/*.txt"

    for item in glob.glob(path):
        file = open(item, 'r')
        name = file.name
        data = file.read()
        data_list.append(name + '\n' + data)
    return data_list


def return_player_match_data(data_list):
    player_data_for_match = []

    for match in data_list:
        number_of_lines = get_number_of_lines_in_file(match)
        afl_round = get_round(match)
        afl_year = get_year(match)
        name = get_url_of_match(match)
        match_id = get_match_id(match)

        for line in range(5, number_of_lines - 1):
            line_data = match.splitlines()[line]
            line_data = [x.strip() for x in line_data.split(',')]
            line_data = [match_id] + [name] + [afl_round] + [afl_year] + line_data
            player_data_for_match.append(line_data)
    return player_data_for_match


match_data_list = get_match_data_list()
player_data = return_player_match_data(match_data_list)
file_name = "fanfooty_match_data_{}.csv".format(timestr)
with open("{}/{}".format(destination, file_name), "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(column_header_names)
    for item in player_data:
        writer.writerow(item)

df_fanfooty_player_raw = pd.read_csv("{}/{}".format(destination, file_name), error_bad_lines=False)
df_fanfooty_player_raw

C:\Users\richardm\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10,32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Fanfooty Match ID,Fanfooty Match URL,Round,Year,Player ID,First Name,Surname,Team,null,DT,...,null12,AF Breakeven,null13,Contested Possessions,Clearances,Clangers,Disposal efficiency,Time on ground,Metres gained,Bench staus
0,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,990020.0,Andrew,Embley,WC,30.0,111.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,230254.0,Adam,Selwood,WC,50.0,107.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,200112.0,Dean,Cox,WC,27.0,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,240016.0,Beau,Waters,WC,26.0,98.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,261911.0,Brad,Ebert,WC,26.0,94.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112259,8053,http://live.fanfooty.com.au/game/matchcentre.h...,R18,2022,1008882.0,Charlie,Ballard,GC,3.0,37.0,...,NaN,NaN,NaN,3.0,0.0,2.0,75.0,81.0,126.0,1.0
112260,8053,http://live.fanfooty.com.au/game/matchcentre.h...,R18,2022,1006087.0,Caleb,Graham,GC,3.0,35.0,...,NaN,NaN,NaN,4.0,0.0,2.0,75.0,96.0,98.0,0.0
112261,8053,http://live.fanfooty.com.au/game/matchcentre.h...,R18,2022,281124.0,Levi,Casboult,GC,9.0,33.0,...,NaN,NaN,NaN,3.0,0.0,2.0,100.0,74.0,43.0,1.0
112262,8053,http://live.fanfooty.com.au/game/matchcentre.h...,R18,2022,297456.0,Nick,Holman,GC,3.0,33.0,...,NaN,NaN,NaN,6.0,0.0,5.0,75.0,88.0,95.0,0.0


### Clean player data

In [9]:
df_fanfooty_player_raw['SC'] = pd.to_numeric(df_fanfooty_player_raw['SC'], errors='coerce')
df_fanfooty_player_raw = df_fanfooty_player_raw.dropna(subset=['SC'])
df_fanfooty_player_raw['SC'] = df_fanfooty_player_raw['SC'].astype('int64')

C:\Users\richardm\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### Identify when players were injured during a match
Fanfooty has amazing "tags" that can be used to identify when a player has been injured during a match

If they have certain tags (e.g. concussed) and score below 80 supercoach points, they are judged as injured.

In [10]:
injured_tags = [
    'sore',
    'injured',
    'longterminjured',
    'concussed',
    'heart',
    'subbed'
]

def get_injured_status(row):
#     if (row['Tag'] in injured_tags or row['Tag 2'] in injured_tags) and row['SC'] < 80:
    if (row['Tag'] in injured_tags or row['Tag 2'] in injured_tags):
        return True
    else:
        return False

df_fanfooty_player_raw['Injured'] = df_fanfooty_player_raw.apply(lambda row: get_injured_status(row), axis=1)
df_fanfooty_player_raw

C:\Users\richardm\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Fanfooty Match ID,Fanfooty Match URL,Round,Year,Player ID,First Name,Surname,Team,null,DT,...,AF Breakeven,null13,Contested Possessions,Clearances,Clangers,Disposal efficiency,Time on ground,Metres gained,Bench staus,Injured
0,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,990020.0,Andrew,Embley,WC,30.0,111.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,230254.0,Adam,Selwood,WC,50.0,107.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,200112.0,Dean,Cox,WC,27.0,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,240016.0,Beau,Waters,WC,26.0,98.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,261911.0,Brad,Ebert,WC,26.0,94.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112259,8053,http://live.fanfooty.com.au/game/matchcentre.h...,R18,2022,1008882.0,Charlie,Ballard,GC,3.0,37.0,...,NaN,NaN,3.0,0.0,2.0,75.0,81.0,126.0,1.0,False
112260,8053,http://live.fanfooty.com.au/game/matchcentre.h...,R18,2022,1006087.0,Caleb,Graham,GC,3.0,35.0,...,NaN,NaN,4.0,0.0,2.0,75.0,96.0,98.0,0.0,False
112261,8053,http://live.fanfooty.com.au/game/matchcentre.h...,R18,2022,281124.0,Levi,Casboult,GC,9.0,33.0,...,NaN,NaN,3.0,0.0,2.0,100.0,74.0,43.0,1.0,False
112262,8053,http://live.fanfooty.com.au/game/matchcentre.h...,R18,2022,297456.0,Nick,Holman,GC,3.0,33.0,...,NaN,NaN,6.0,0.0,5.0,75.0,88.0,95.0,0.0,False


## 3. Match results/fixture

### Get fixture data from Fanfooty website

In [11]:
url = "http://www.fanfooty.com.au/resource/draw.php"
headers = ["FanFooty draw ID", "year", "competition", "round", "gametime (AET)", "day", "home team", "away team", "ground", "timeslot", "TV coverage", "home supergoals", "home goals", "home behinds", "home points", "away supergoals", "away goals", "away behinds", "away points", "match status"]
data_list = []


res = requests.get(url, headers=url_headers, verify=False)
display(res)
if res.text == 'Too many connections':
    raise
        
split_rows_list = res.text.split('"\r\n')
for row in split_rows_list:
    field_list = row.split(',')
    data_list.append(field_list)
    
df_fixture = pd.DataFrame(data_list, columns=headers)

for i, col in enumerate(df_fixture.columns):
    df_fixture.iloc[:, i] = df_fixture.iloc[:, i].str.replace('"', '')
    
df_fixture['FanFooty draw ID'] = pd.to_numeric(df_fixture['FanFooty draw ID'])
df_fixture

C:\Users\richardm\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.fanfooty.com.au'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


<Response [200]>

,FanFooty draw ID,year,competition,round,gametime (AET),day,home team,away team,ground,timeslot,TV coverage,home supergoals,home goals,home behinds,home points,away supergoals,away goals,away behinds,away points,match status
0,1006.0,1993,HA,1,1993-03-26 20:08:00,Friday,Western Bulldogs,Collingwood,MCG,N,,,13,17,95,,17,13,115,Full Time
1,1000.0,1993,HA,1,1993-03-27 14:00:00,Saturday,North Melbourne,Brisbane Bears,MCG,D,,,24,22,166,,22,11,143,Full Time
2,1001.0,1993,HA,1,1993-03-27 14:00:00,Saturday,Carlton,Fitzroy,Princes Park,D,,,17,10,112,,17,16,118,Full Time
3,1002.0,1993,HA,1,1993-03-27 14:00:00,Saturday,Hawthorn,Naarm,Waverley,D,,,13,15,93,,11,4,70,Full Time
4,1005.0,1993,HA,1,1993-03-27 14:08:00,Saturday,Geelong,St Kilda,Kardinia,D,,,20,16,136,,16,16,112,Full Time
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6162,8098.0,2022,HA,23,2022-08-19 17:20:00,Friday,St Kilda,Sydney,Docklands,C,TBC,,,,,,,,,
6163,8094.0,2022,HA,23,2022-08-19 19:25:00,Friday,GWS,Fremantle,Manuka,C,TBC,,,,,,,,,
6164,8095.0,2022,HA,23,2022-08-19 19:40:00,Friday,Hawthorn,Western Bulldogs,Launceston,C,TBC,,,,,,,,,
6165,8090.0,2022,HA,23,2022-08-19 19:50:00,Friday,Brisbane Lions,Naarm,Gabba,C,TBC,,,,,,,,,


### Clean Fixture data

In [12]:
df_fixture = df_fixture.loc[df_fixture['match status'] == ' Full Time']
df_fixture['FanFooty draw ID'] = df_fixture['FanFooty draw ID'].astype('int')
df_fixture

C:\Users\richardm\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,FanFooty draw ID,year,competition,round,gametime (AET),day,home team,away team,ground,timeslot,TV coverage,home supergoals,home goals,home behinds,home points,away supergoals,away goals,away behinds,away points,match status
0,1006,1993,HA,1,1993-03-26 20:08:00,Friday,Western Bulldogs,Collingwood,MCG,N,,,13,17,95,,17,13,115,Full Time
1,1000,1993,HA,1,1993-03-27 14:00:00,Saturday,North Melbourne,Brisbane Bears,MCG,D,,,24,22,166,,22,11,143,Full Time
2,1001,1993,HA,1,1993-03-27 14:00:00,Saturday,Carlton,Fitzroy,Princes Park,D,,,17,10,112,,17,16,118,Full Time
3,1002,1993,HA,1,1993-03-27 14:00:00,Saturday,Hawthorn,Naarm,Waverley,D,,,13,15,93,,11,4,70,Full Time
4,1005,1993,HA,1,1993-03-27 14:08:00,Saturday,Geelong,St Kilda,Kardinia,D,,,20,16,136,,16,16,112,Full Time
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6116,8049,2022,HA,18,2022-07-16 19:25:00,Saturday,Carlton,Geelong,MCG,N,TBC,,8,7,55,,12,13,85,Full Time
6117,8050,2022,HA,18,2022-07-16 19:30:00,Saturday,Fremantle,Sydney,Perth,N,TBC,,9,11,65,,11,16,82,Full Time
6118,8051,2022,HA,18,2022-07-17 13:10:00,Sunday,Hawthorn,West Coast,MCG,D,TBC,,15,12,102,,12,5,77,Full Time
6119,8052,2022,HA,18,2022-07-17 15:20:00,Sunday,Naarm,Port Adelaide,Alice Springs,D,TBC,,12,11,83,,10,9,69,Full Time


# Get standard team name

In [13]:
df_team_names = pd.read_csv("inputs/all_sources_team_names.csv")
df_fixture = df_fixture.merge(df_team_names[['fanfooty_team_name', 'fanfooty_team_name_mid']], left_on='home team' ,right_on='fanfooty_team_name_mid', how='left')
df_fixture = df_fixture.rename(columns={"fanfooty_team_name": "home_team_short"})
del df_fixture['fanfooty_team_name_mid']
df_fixture = df_fixture.merge(df_team_names[['fanfooty_team_name', 'fanfooty_team_name_mid']], left_on='away team' ,right_on='fanfooty_team_name_mid', how='left')
df_fixture = df_fixture.rename(columns={"fanfooty_team_name": "away_team_short"})
del df_fixture['fanfooty_team_name_mid']
df_fixture.drop(df_fixture.tail(1).index,inplace=True)
df_fixture

,FanFooty draw ID,year,competition,round,gametime (AET),day,home team,away team,ground,timeslot,...,home goals,home behinds,home points,away supergoals,away goals,away behinds,away points,match status,home_team_short,away_team_short
0,1006,1993,HA,1,1993-03-26 20:08:00,Friday,Western Bulldogs,Collingwood,MCG,N,...,13,17,95,,17,13,115,Full Time,WB,CO
1,1000,1993,HA,1,1993-03-27 14:00:00,Saturday,North Melbourne,Brisbane Bears,MCG,D,...,24,22,166,,22,11,143,Full Time,NM,NaN
2,1001,1993,HA,1,1993-03-27 14:00:00,Saturday,Carlton,Fitzroy,Princes Park,D,...,17,10,112,,17,16,118,Full Time,CA,NaN
3,1002,1993,HA,1,1993-03-27 14:00:00,Saturday,Hawthorn,Naarm,Waverley,D,...,13,15,93,,11,4,70,Full Time,HW,ME
4,1005,1993,HA,1,1993-03-27 14:08:00,Saturday,Geelong,St Kilda,Kardinia,D,...,20,16,136,,16,16,112,Full Time,GE,SK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6047,8048,2022,HA,18,2022-07-16 16:35:00,Saturday,North Melbourne,Richmond,Docklands,T,...,14,8,92,,11,22,88,Full Time,NM,RI
6048,8049,2022,HA,18,2022-07-16 19:25:00,Saturday,Carlton,Geelong,MCG,N,...,8,7,55,,12,13,85,Full Time,CA,GE
6049,8050,2022,HA,18,2022-07-16 19:30:00,Saturday,Fremantle,Sydney,Perth,N,...,9,11,65,,11,16,82,Full Time,FR,SY
6050,8051,2022,HA,18,2022-07-17 13:10:00,Sunday,Hawthorn,West Coast,MCG,D,...,15,12,102,,12,5,77,Full Time,HW,WC


### Get the total SuperCoach and AFL Fantasy scores for each team, for every match

In [14]:
# Create a summary to get the total SC and AF points for each match
df_match_summary = pd.pivot_table(df_fanfooty_player_raw, index=['Fanfooty Match ID'], values=['SC'], columns=['Team'], aggfunc=np.sum)
df_match_summary = df_match_summary.reset_index()
headings = [x[1] for x in df_match_summary.columns]
headings[0] = 'Fanfooty Match ID'
df_match_summary.columns = headings
df_match_summary

,Fanfooty Match ID,AD,BL,CA,CO,ES,FR,GC,GE,HW,ME,NM,PA,RI,SK,SY,WB,WC,WS
0,3425,NaN,NaN,NaN,NaN,1568.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1739.0,NaN
1,3426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1504.0,NaN,NaN,NaN,1797.0,NaN,NaN,NaN
2,3427,1513.0,NaN,1826.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3428,NaN,NaN,NaN,1873.0,NaN,NaN,NaN,NaN,1447.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3429,NaN,1781.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1545.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2526,8049,NaN,NaN,1614.0,NaN,NaN,NaN,NaN,1683.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2527,8050,NaN,NaN,NaN,NaN,NaN,1558.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1741.0,NaN,NaN,NaN
2528,8051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1653.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1650.0,NaN
2529,8052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1629.0,NaN,1671.0,NaN,NaN,NaN,NaN,NaN,NaN


### Join each team's SC total to fixture data + save file

In [15]:
def get_sc_total(row, team_short):
    match_id = row['FanFooty draw ID']
    team_name = row[team_short]
    if match_id >= 3425:
        sc_total = df_match_summary.loc[df_match_summary['Fanfooty Match ID'] == match_id, team_name]
        if len(sc_total) > 0: 
            return sc_total.values[0]

df_fixture['Home SC total'] = df_fixture.apply(lambda row: get_sc_total(row, 'home_team_short'), axis=1)
df_fixture['Away SC total'] = df_fixture.apply(lambda row: get_sc_total(row, 'away_team_short'), axis=1)
df_fixture.to_csv('{}/fanfooty_fixture.csv'.format(destination))
df_fixture

,FanFooty draw ID,year,competition,round,gametime (AET),day,home team,away team,ground,timeslot,...,home points,away supergoals,away goals,away behinds,away points,match status,home_team_short,away_team_short,Home SC total,Away SC total
0,1006,1993,HA,1,1993-03-26 20:08:00,Friday,Western Bulldogs,Collingwood,MCG,N,...,95,,17,13,115,Full Time,WB,CO,NaN,NaN
1,1000,1993,HA,1,1993-03-27 14:00:00,Saturday,North Melbourne,Brisbane Bears,MCG,D,...,166,,22,11,143,Full Time,NM,NaN,NaN,NaN
2,1001,1993,HA,1,1993-03-27 14:00:00,Saturday,Carlton,Fitzroy,Princes Park,D,...,112,,17,16,118,Full Time,CA,NaN,NaN,NaN
3,1002,1993,HA,1,1993-03-27 14:00:00,Saturday,Hawthorn,Naarm,Waverley,D,...,93,,11,4,70,Full Time,HW,ME,NaN,NaN
4,1005,1993,HA,1,1993-03-27 14:08:00,Saturday,Geelong,St Kilda,Kardinia,D,...,136,,16,16,112,Full Time,GE,SK,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6047,8048,2022,HA,18,2022-07-16 16:35:00,Saturday,North Melbourne,Richmond,Docklands,T,...,92,,11,22,88,Full Time,NM,RI,1695.0,1604.0
6048,8049,2022,HA,18,2022-07-16 19:25:00,Saturday,Carlton,Geelong,MCG,N,...,55,,12,13,85,Full Time,CA,GE,1614.0,1683.0
6049,8050,2022,HA,18,2022-07-16 19:30:00,Saturday,Fremantle,Sydney,Perth,N,...,65,,11,16,82,Full Time,FR,SY,1558.0,1741.0
6050,8051,2022,HA,18,2022-07-17 13:10:00,Sunday,Hawthorn,West Coast,MCG,D,...,102,,12,5,77,Full Time,HW,WC,1653.0,1650.0


### Add home/away + opposition columns to player data

In [16]:
def get_opposition(row):
    match_id = row['Fanfooty Match ID']
    team = row['Team']
    home_team = df_fixture.loc[(df_fixture['FanFooty draw ID'] == match_id) & (df_fixture['home_team_short'] == team)]
    away_team = df_fixture.loc[(df_fixture['FanFooty draw ID'] == match_id) & (df_fixture['away_team_short'] == team)]
    if len(home_team) > 0:
        return home_team['away_team_short'].values[0]
    if len(away_team) > 0:
        return away_team['home_team_short'].values[0]
    

df_fanfooty_player_raw['Opposition'] = df_fanfooty_player_raw.apply(lambda row: get_opposition(row), axis=1)
df_fanfooty_player_raw.to_csv("{}/{}".format(destination, file_name))
df_fanfooty_player_raw

C:\Users\richardm\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,Fanfooty Match ID,Fanfooty Match URL,Round,Year,Player ID,First Name,Surname,Team,null,DT,...,null13,Contested Possessions,Clearances,Clangers,Disposal efficiency,Time on ground,Metres gained,Bench staus,Injured,Opposition
0,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,990020.0,Andrew,Embley,WC,30.0,111.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,ES
1,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,230254.0,Adam,Selwood,WC,50.0,107.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,ES
2,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,200112.0,Dean,Cox,WC,27.0,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,ES
3,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,240016.0,Beau,Waters,WC,26.0,98.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,ES
4,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,261911.0,Brad,Ebert,WC,26.0,94.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,ES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112259,8053,http://live.fanfooty.com.au/game/matchcentre.h...,R18,2022,1008882.0,Charlie,Ballard,GC,3.0,37.0,...,NaN,3.0,0.0,2.0,75.0,81.0,126.0,1.0,False,None
112260,8053,http://live.fanfooty.com.au/game/matchcentre.h...,R18,2022,1006087.0,Caleb,Graham,GC,3.0,35.0,...,NaN,4.0,0.0,2.0,75.0,96.0,98.0,0.0,False,None
112261,8053,http://live.fanfooty.com.au/game/matchcentre.h...,R18,2022,281124.0,Levi,Casboult,GC,9.0,33.0,...,NaN,3.0,0.0,2.0,100.0,74.0,43.0,1.0,False,None
112262,8053,http://live.fanfooty.com.au/game/matchcentre.h...,R18,2022,297456.0,Nick,Holman,GC,3.0,33.0,...,NaN,6.0,0.0,5.0,75.0,88.0,95.0,0.0,False,None


In [17]:
"{}/{}".format(destination, file_name)

'exports/scrape_20220720-171354/fanfooty_match_data_20220720-171354.csv'